# Assignment 1: Street View {-}
The [Street View House Numbers Dataset](https://www.openml.org/d/41081) contains 32-by-32 RGB images centered around a single digit of a house number appearing in Google Street View. Many of the images do contain some distractors at the sides. It consists of 10 classes, 1 for each digit. Digit '1' has label 1, '9' has label 9 and '0' has label 10. Your goal is to build a model that recognizes the correct digit.

In [ ]:
%matplotlib inline
#!pip install openml # Uncomment to run in Google Colab
import openml as oml
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Download Streetview data. Takes a while the first time.
SVHN = oml.datasets.get_dataset(41081)
X, y, cats, attrs = SVHN.get_data(dataset_format='array',
    target=SVHN.default_target_attribute)

In [ ]:
# Plots image. Use grayscale=True for plotting grayscale images
def plot_images(X, y, grayscale=False):
    fig, axes = plt.subplots(1, len(X),  figsize=(10, 5))
    if grayscale:
        [ax.imshow(X[n].reshape(32, 32)/255, cmap='gray')
         for n,ax in enumerate(axes)]
    else:
        [ax.imshow(X[n].reshape(32, 32, 3)/255) for n,ax in enumerate(axes)]
    [ax.set_title((y[n]+1)) for n,ax in enumerate(axes)]
    [ax.axis('off') for ax in axes]
plot_images(X[0:5], y[0:5]);

> **Instructions and advice (remove before submission)**
>  
> * Answer the questions in this notebook, including the code, results, and discussions. 
> * Add precise explanations when interpreting the results of your experiments. Use markdown cells for this.
> * Submit both this notebook and a PDF through Canvas. To create the PDF, see File > Export Notebook as PDFs.
> * Submit only one notebook and one PDF per team. In Jupyter Lab, you can copy-paste cells (with results) into a single notebook. 
> * Keep the PDF below 20 pages. Remove these instructions and the general advise below in the final PDF.
> * Avoid all(!) unnecessary outputs. Only output the answers to the questions. Add ';' behind lines that > generate output to suppress unnecessary output.
> * All tasks can be completed with scikit-learn. You are allowed (but not required) to use other tools as well.
> * Training models can take time. Make sure to start computations well in advance of the deadline. 
> * Be efficient. Don't try every model on all the data at once. Test your code on a small part of the data (even just 1%) until you are sure that there are no more bugs or other issues.
> * Observe how long it takes to train different models on a small part of your data to learn whether the models are fast enough for what you want to do.
> * You are allowed to run code elsewhere (e.g. in Python scripts) and store the results on file. If so, copy the code in this notebook and load/visualize/discuss the results here as well.
> * On [Google Colab](https://colab.research.google.com/) you can run the notebook in the cloud.

### Team details {-}
Add the names of all team members and a short description of each member's contributions.   
TEAM MEMBER 1 = Yme van der Linden   
TEAM MEMBER 2 = Angela Cha  
TEAM MEMBER 3 = Perry van Asseldonk   
TEAM MEMBER 4 = Irfan Ahmed  
TEAM MEMBER 5 = Merel Ypinga  

## 1. Evaluate different models (15 points) {-}

In [ ]:
#Take a stratified 10% subsample of the data. 
#Plot the distribution of the classes (as a histogram) for both the original data and the subsample.

#Use train_test_split in a "clever" way to generate 10% stratified sub-sample:
X_90_percent, X_10_percent, y_90_percent, y_10_percent = train_test_split(X, y, test_size=0.1, stratify=y) 


plt.hist(y_10_percent, bins=10, histtype='bar', edgecolor='black')         #Histogram of Sub-sample
plt.title("Distribution of sub-sample y")
plt.xlabel("Different classes (0-9, 0 is coded as 10, 1 is coded as 1) ")
plt.ylabel("Count")
plt.show();

plt.hist(y, bins=10, histtype='bar', edgecolor='black')                    #Histogram of original data
plt.title("Distribution of y (full dataset)")
plt.xlabel("Different classes (0-9, 0 is coded as 10, 1 is coded as 1) ")
plt.ylabel("Count")
plt.show();

In [ ]:
#Evaluate k-Nearest Neighbors, using default hyperparameter settings. Use cross-validation with 3 folds, 
#output the training accuracy and test accuracy including the standard deviations

#Initialize default knn classifier
knn = KNeighborsClassifier()
#Use cross_validate with regular CV (As sample is already stratified, and we were not asked to use StratifiedKfold)
scores = cross_validate(knn, X_10_percent, y_10_percent, cv=3, scoring=['accuracy'], return_train_score=True, random_state =47)

#Report training accuracy + std, testing accuracy + std
print("Training accuracy of models {}".format(scores['train_accuracy']))            #Training accuracy for 3 folds: [0.56695889 0.566853   0.57104031] 
print("Standard deviation of training accuracies are {}".format(scores['train_accuracy'].std())) #std over 3 folds: 0.001949440450662831
print("Testing accuracy of models {}".format(scores['test_accuracy']))               #Testing accuracy for 3 folds: [0.35194688 0.34833837 0.34361766]
print("Standard deviation of Training accuracies {}".format(scores['test_accuracy'].std()))      #std over 3 folds: 0.0034104761232712894

In [ ]:
#Evaluate Logistic regression, using default hyperparameter settings. Use cross-validation with 3 folds, 
#output the training accuracy and test accuracy including the standard deviations

#Initialize default logistic regression 
#(it infers that we are using a multi-classifying problem instead of binary, as multi-class is set to auto by default)
logisticReg = LogisticRegression()

#Use cross_validate with regular CV (As sample is already stratified, and we were not asked to use StratifiedKfold)
scores = cross_validate(logisticReg, X_10_percent, y_10_percent, cv=3, scoring=['accuracy'], return_train_score=True, random_state = 47)

#Report training accuracy + std, testing accuracy + std
print("Training accuracy of models {}".format(scores['train_accuracy']))            #Training accuracy for 3 folds: [0.75226723 0.74935791 0.73622226] 
print("Standard deviation of training accuracies are {}".format(scores['train_accuracy'].std())) #std over 3 folds: 0.006979738688354459
print("Testing accuracy of models {}".format(scores['test_accuracy']))               #Testing accuracy for 3 folds: [0.16752188 0.16616314 0.17906836]
print("Standard deviation of Training accuracies {}".format(scores['test_accuracy'].std()))      #std over 3 folds: 0.00578995276573481

In [ ]:
#Evaluate Linear SVM, using default hyperparameter settings. Use cross-validation with 3 folds, 
#output the training accuracy and test accuracy including the standard deviations

#Initialize default linear support vector machine
svc = LinearSVC()

#Use cross_validate with regular CV (As sample is already stratified, and we were not asked to use StratifiedKfold)
scores = cross_validate(svc, X_10_percent, y_10_percent, cv=3, scoring=['accuracy'], return_train_score=True, random_state =47)

#Report training accuracy + std, testing accuracy + std
print("Training accuracy of models {}".format(scores['train_accuracy']))            #Training accuracy for 3 folds: [0.2028416  0.22541169 0.41869244]
print("Standard deviation of training accuracies are {}".format(scores['train_accuracy'].std())) #std over 3 folds: 0.09687244723712729
print("Testing accuracy of models {}".format(scores['test_accuracy']))               #Testing accuracy for 3 folds: [0.14941141 0.19365559 0.16333938]
print("Standard deviation of Training accuracies {}".format(scores['test_accuracy'].std()))      #std over 3 folds: 0.018471023013982568

## Evaluation of results
Discuss the results. Are they what you expected? Are the model over/underfitting? What does the standard deviation tell you? Do you think you can get better results with more training data?
Results: see comments



## 2. Learning curves (20 points) {-}
- Start with a 1% stratified subsample of the original data. Train all 3 classifiers and record both the accuracy (same as above) and the [training time](https://pythonhow.com/measure-execution-time-python-code/).  
- Repeatedly **add** 1% more statified data (don't take a completely new 2% subsample). Measure accuracy and training time again.
- Continue to at least 10% of the original data. Plot the results as a line graph, with training set size on the x-axis and accuracy and training time, respectively, on the y-axes.
    - If the algorithms are sufficiently fast, you can continue to 100% as well.

In [ ]:
test_accuracy_list = []
test_accuracy_std_list = []
time_execution_list = []
training_percentages = [1,2,3,4,5,6,7,8,9,10] #Values for x-axis of plots 

knn = KNeighborsClassifier()
svc = LinearSVC()
logisticReg = LogisticRegression()


for i in range(1,11):
    """We use time.time() instead of timeit() because time.time() lets us time a block of code (So in this case an)
       iteration of a for-loop, whereas timeit only times complete scripts, so we would have to run all methods 10 times
       we increase the training data by 992 samples at a time (Which is 1% of the 10% subsample, which has 9920 data rows)
       Then we append the scores and training times to lists, which can later easily be plotted. Fill in the appropriate 
       classifier in the cross_validate method (knn, svc or logisticReg, declared above).
       """
    #NOTE FOR MY GROUP: PLOTS GENERATED BY THIS CODE CAN BE FOUND IN THE PLOTS Q2 FOLDER ON GITHUB
    start = time.time()  
    scores = cross_validate(knn, X_10_percent[:i*992], y_10_percent[0:i*992], cv=3, scoring=['accuracy'], n_jobs=-1)
    test_accuracy_list.append(scores['test_accuracy'])
    test_accuracy_std_list.append(scores['test_accuracy'].std())
    stop = time.time()
    duration = start-stop
    time_execution_list.append(duration)
    print("Training complete on {}% subsample of data".format(i))


plt.subplot(2,1,1)
plt.plot(training_percentages, test_accuracy_list, '-o')
plt.title('Testing accuracy and training times plot')
plt.xlabel("Percentage of data used")
plt.ylabel("Testing accuracy")

plt.subplot(2,1,2)
plt.plot(training_percentages,time_execution_list , '-o')
plt.xlabel("Percentage of data used")
plt.ylabel("Execution time in seconds")
plt.show()

- Discuss your results. Which algorithms are fast/slow? Which require more data to get better results? Which ones tend to overfit? Which ones would benefit from more data? #TODO

## 3. Data representation (20 points) {-}
* Convert the images to grayscale (use the function below). Check whether the conversion is correct by plotting a few of the converted images.

In [ ]:
def rgb2gray(X, dim=32):
    return np.squeeze(np.expand_dims(np.dot(X.reshape(len(X), dim * dim, 3), [0.2990, 0.5870, 0.1140]), axis=3))

X_gray = rgb2gray(X)

def plot_images(X, y):
    fig, axes = plt.subplots(1, len(X),  figsize=(10, 5))
    [ax.imshow(X[n].reshape(32, 32)/255, cmap='gray') for n,ax in enumerate(axes)]
    [ax.set_title((y[n]+1)) for n,ax in enumerate(axes)]
    [ax.axis('off') for ax in axes]
    plt.show()

plot_images(X_gray[0:5], y[0:5]) #Conversion is correct

* Is the grayscale representation better than the color representation? Does it yield better models? Why (not)? Run the necessary experiments and discuss. For instance, compare the performance of models trained on both representations.

In [ ]:
# To check the difference in performance, we compare the results of different models with the results obtained in question 1.
# Since results of Q1 were already obtained, we simply run the same models as we did in Q1, but now on the grayscale data.
# Alongside test accuracy we also generate the training accuracy and standard deviations again as they could provide us with
# Some useful insights about the training process.
# Also generate a new test split
X_gray_90_percent, X_gray_10_percent, y_90_percent, y_10_percent = train_test_split(X_gray, y, test_size=0.1, stratify=y, random_state=47)

In [ ]:
#Initialize default knn classifier
knn = KNeighborsClassifier()
#Use cross_validate with regular CV (As sample is already stratified, and we were not asked to use StratifiedKfold)
scores = cross_validate(knn, X_gray_10_percent, y_10_percent, cv=3, scoring=['accuracy'], return_train_score=True)

#Report training accuracy + std, testing accuracy + std
print("Training accuracy of models {}".format(scores['train_accuracy']))            #Training accuracy for 3 folds: [0.58388755 0.58951503 0.58417636]
print("Standard deviation of training accuracies are {}".format(scores['train_accuracy'].std())) #std over 3 folds: 0.002587437567413254
print("Testing accuracy of models {}".format(scores['test_accuracy']))               #Testing accuracy for 3 folds: [0.38182916 0.37190332 0.37295826]
print("Standard deviation of Training accuracies {}".format(scores['test_accuracy'].std()))      #std over 3 folds: 0.004451316268125431

In [ ]:
#Initialize default support vector machine
svc = LinearSVC()
#Use cross_validate with regular CV (As sample is already stratified, and we were not asked to use StratifiedKfold)
scores = cross_validate(svc, X_gray_10_percent, y_10_percent, cv=3, scoring=['accuracy'], return_train_score=True)

#Report training accuracy + std, testing accuracy + std
print("Training accuracy of models {}".format(scores['train_accuracy']))            #Training accuracy for 3 folds: [0.20133011 0.12161958 0.26105994]
print("Standard deviation of training accuracies are {}".format(scores['train_accuracy'].std())) #std over 3 folds: 0.05712076503282101
print("Testing accuracy of models {}".format(scores['test_accuracy']))               #Testing accuracy for 3 folds: [0.13975249 0.08851964 0.13883848]
print("Standard deviation of Training accuracies {}".format(scores['test_accuracy'].std()))      #std over 3 folds: 0.023938871417926368


In [ ]:
#Initialize default logistic regression
logisticReg = LogisticRegression()
#Use cross_validate with regular CV (As sample is already stratified, and we were not asked to use StratifiedKfold)
scores = cross_validate(logisticReg, X_gray_10_percent, y_10_percent, cv=3, scoring=['accuracy'], return_train_score=True)

#Report training accuracy + std, testing accuracy + std
print("Training accuracy of models {}".format(scores['train_accuracy']))            #Training accuracy for 3 folds:
print("Standard deviation of training accuracies are {}".format(scores['train_accuracy'].std())) #std over 3 folds:
print("Testing accuracy of models {}".format(scores['test_accuracy']))               #Testing accuracy for 3 folds:
print("Standard deviation of Training accuracies {}".format(scores['test_accuracy'].std()))      #std over 3 folds:

* Afterwards, normalize the data by substracting the mean of the pixel values from every image and dividing by the standard deviation of very image. 
    * Note: Do this for every image independently. Don't compute means or standard deviations over the entire training/test set
    * Again, check whether you have done it correctly by plotting some of the converted images.
    * Is the new representation better? Why (not)? Run the necessary experiments and discuss.
* Finally, run [PCA (Principal Component Analysis)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) on the data and see if that gives you a better (lower-dimensional) representation. Note that you need to use a pipeline here to avoid data leakage between your training and test set (see Tutorial 3).
    * Try different numbers of components or use a solver to guess the optimal number of components
    * Is the new representation better? Why (not)? Run the necessary experiments and discuss.

## 4. Regularization (20 points) {-}
- Tune the algorithm's main regularization hyperparameter on the 10% stratified subsample using the normalized grayscale representation. These are the misclassification cost in SVMs (C), the regularization parameter in logistic regression (C), and the number of neighbors (n_neighbors) in kNN. 
- We expect the optimum for the C parameters to lie in $[10^{-12},10^{12}] and for n_neighbors between 1 and 50. C should be varied logarithmically, n_neighbors linearly. 
- Run a grid search over these ranges, using 3-fold cross-valdation, and plot both the training and test error. Start with a coarse grid (e.g. 5 values per hyperparameter), use a finer grid for the 'interesting' regions (where performance changes) if training time allows.
- Plot the effect of every one of the hyperparameters (both the training and test score). Discuss the plots. For which values is the model over/underfitting? Is there a sweet spot?
- Evaluate the optimized models (the best hyperparameter settings) on the rest of the (unseen) data and discuss your findings. Do the results generalize? If not, what can/ should you do? Discuss. Run additional experiments if it helps you confirm your claims.

## 5. Interpreting misclassifications (10 points) {-}
- Chances are that your best model is not 100% accurate. Plot a few of the misclassified examples and their predicted and correct labels. Discuss. Are these indeed 'hard' examples that are easy to get wrong?
- Plot the confusion matrix. Which digits are often confused with each other? Which digits are easier/harder to classify? Discuss.

## 6. Interpreting models (15 points) {-}
- Try to understand what the linear models have learned by looking at their model parameters. Are there features (pixels) that are more important than others? What does that tell you?
- Visualize the model parameters of the best (tuned) models as a 32x32 matrix, for each class individually. This means training 10 models in a one-vs-all fashion. Does this help you understand what the model is paying attention to?
- Do the same for an underfitted and an overfitted version of logistic regression. What changes? Is the model paying attention to different parts of the image now?

Good luck!